In [ ]:
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
import tensorflow.keras as keras
from keras import Model
from keras.applications.densenet import DenseNet121
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
path = 'archive/'
dataset_path = pathlib.Path(path)
dirs_names = os.listdir(dataset_path) # list content of dataset
dirs_names

['images', 'myntradataset', 'styles.csv']

In [ ]:
styles_df = pd.read_csv(path + "styles.csv", nrows=40000, error_bad_lines=False) # Read n product and drop bad lines 
styles_df['image'] = styles_df.apply(lambda x: str(x['id']) + ".jpg", axis=1) # Make image column contains (id.jpg)
styles_df.head()

C:\Users\samis\AppData\Local\Temp\ipykernel_15564\349633451.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  styles_df = pd.read_csv(path + "styles.csv", nrows=40000, error_bad_lines=False) # Read n product and drop bad lines
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping 

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [ ]:
#plt.figure(figsize=(7,20))
#styles_df.articleType.value_counts().sort_values().plot(kind='barh')

In [ ]:
#In our case, we don't need every type of clothes

top_25 = styles_df['articleType'].value_counts().sort_values(ascending=False).head(25)
top_25_list = top_25.index
print(top_25_list)


styles_df_processed = styles_df.loc[styles_df['articleType'].isin(top_25_list), :]
# 
top5_each_Type = pd.DataFrame(columns=styles_df.columns)

#we take only n elements of each type to be compared -> to reduce computation time when we recommend
for X in top_25_list:
    top5_each_Type = top5_each_Type.append(styles_df.loc[styles_df['articleType'] == X, :].head(20))

print(top5_each_Type['id'].tolist())



Index(['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes',
       'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses', 'Wallets',
       'Flip Flops', 'Sandals', 'Briefs', 'Belts', 'Backpacks', 'Socks',
       'Formal Shoes', 'Perfume and Body Mist', 'Jeans', 'Shorts', 'Trousers',
       'Flats', 'Dresses', 'Bra'],
      dtype='object')
[53759, 1855, 7990, 4729, 3954, 5891, 38630, 10866, 15528, 2288, 18237, 13419, 5865, 29584, 9694, 8322, 7964, 17240, 3365, 10065, 15970, 30805, 26960, 12369, 37812, 56825, 9452, 4943, 9660, 15984, 59297, 7158, 16392, 6617, 22395, 12500, 16530, 7167, 12190, 14055, 9204, 39988, 22198, 29570, 26538, 49495, 19311, 49461, 23849, 24250, 15517, 4140, 6425, 42270, 32751, 39943, 22705, 50741, 20856, 19772, 59263, 30039, 29928, 17429, 51658, 23278, 44984, 11188, 45258, 44970, 22950, 13841, 8110, 43190, 32335, 56670, 36795, 40527, 45869, 4716, 3168, 6628, 33822, 42089, 23882, 34835, 12993, 42042, 17416, 12958, 46086, 54543, 4524, 36137, 8913, 5402, 433

C:\Users\samis\AppData\Local\Temp\ipykernel_15564\74402011.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5_each_Type = top5_each_Type.append(styles_df.loc[styles_df['articleType'] == X, :].head(20))
C:\Users\samis\AppData\Local\Temp\ipykernel_15564\74402011.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5_each_Type = top5_each_Type.append(styles_df.loc[styles_df['articleType'] == X, :].head(20))
C:\Users\samis\AppData\Local\Temp\ipykernel_15564\74402011.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5_each_Type = top5_each_Type.append(styles_df.loc[styles_df['articleType'] == X, :].head(20))
C:\Users\samis\AppData\Local\Temp\ipykernel_15564\74402011.py:14: FutureWarning: The frame.append method i

In [ ]:
from collections import Counter

def find_duplicate(lst):
    # Create a Counter object with the list
    element_count = Counter(lst)

    # Iterate over the Counter and print the elements with a count greater than 1
    for elem, count in element_count.items():
        if count > 1:
            print(f"Element {elem} is not unique, it appears {count} times.")

find_duplicate(top5_each_Type['id'].tolist())

In [ ]:

"""
j=0
print(path)
for i in (top5_each_Type['id'].tolist()):
    plt.subplot(6, 10, j+1)
    try:
        cloth_img =  mpimg.imread(path + 'images/'+ str(i) +".jpg")
    except FileNotFoundError:
        print("Could not find the specified file : ",path + 'images/'+ str(i))
        continue
    except Exception as e:
        print("An error occurred:", e)
    plt.imshow(cloth_img)
    plt.axis("off")
    j+=1
plt.title("Recommended images",loc='left')
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()"""


'\nj=0\nprint(path)\nfor i in (top5_each_Type[\'id\'].tolist()):\n    plt.subplot(6, 10, j+1)\n    try:\n        cloth_img =  mpimg.imread(path + \'images/\'+ str(i) +".jpg")\n    except FileNotFoundError:\n        print("Could not find the specified file : ",path + \'images/\'+ str(i))\n        continue\n    except Exception as e:\n        print("An error occurred:", e)\n    plt.imshow(cloth_img)\n    plt.axis("off")\n    j+=1\nplt.title("Recommended images",loc=\'left\')\nplt.subplots_adjust(wspace=-0.5, hspace=1)\nplt.show()'

In [ ]:
print(top5_each_Type)

         id gender masterCategory subCategory articleType baseColour  season  \
4     53759    Men        Apparel     Topwear     Tshirts       Grey  Summer   
5      1855    Men        Apparel     Topwear     Tshirts       Grey  Summer   
27     7990    Men        Apparel     Topwear     Tshirts  Navy Blue    Fall   
31     4729   Boys        Apparel     Topwear     Tshirts      Green  Summer   
39     3954  Women        Apparel     Topwear     Tshirts       Pink  Summer   
...     ...    ...            ...         ...         ...        ...     ...   
1334  45097  Women        Apparel   Innerwear         Bra       Pink  Winter   
1422  56276  Women        Apparel   Innerwear         Bra       Skin  Summer   
1457  56420  Women        Apparel   Innerwear         Bra      White  Winter   
1575  57350  Women        Apparel   Innerwear         Bra      Black  Winter   
1579  56282  Women        Apparel   Innerwear         Bra      Black  Summer   

        year   usage                   

In [ ]:
img_width, img_height, chnls = 100, 100, 3

#VGG16 is a already trained model that we fine tune here
from tensorflow.keras.applications import VGG16

vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(img_width, img_height, chnls))
vgg16.trainable=False
vgg16_model = keras.Sequential([vgg16, GlobalMaxPooling2D()])




In [ ]:
def img_path(img):
    """ Take image name(id) and return the complete path of it """
    return path + 'images/' + img

def predict(model, img_name):
    """ Load and preprocess image then make prediction """
    # Reshape
    img = keras.utils.load_img(img_path(img_name), target_size=(img_width, img_height))
    # img to Array
    img = keras.utils.img_to_array(img)
    # Expand Dim (1, w, h)
    img = np.expand_dims(img, axis=0)
    # Pre process Input
    img = preprocess_input(img)
    return model.predict(img)

In [ ]:


def get_embeddings(df, model):
    """ Return a dataframe contains images features """
    df_copy = df
    df_embeddings = df_copy['image'].apply(lambda x: predict(vgg16_model, x).reshape(-1))
    df_embeddings = df_embeddings.apply(pd.Series)
    return df_embeddings

df_embeddings = get_embeddings(top5_each_Type, vgg16_model)



1/1 [==============================] - 0s 58ms/step


In [ ]:
from PIL import Image

image_name = "IMG_0546.jpg"
new_image_name = "resized_"+image_name
# Open the image
im=Image.open(new_image_name) 
# Resize the image
im_resized = im.resize((150, 150))

# Save the resized image
im_resized.save("archive/images/"+new_image_name)
im_resized.save(new_image_name)

In [ ]:



sample_image = predict(vgg16_model, new_image_name)
df_sample_image = pd.DataFrame(sample_image) 
sample_similarity = linear_kernel(df_sample_image, df_embeddings)
def get_similarity(model):
    """ Get similarity of custom image """
    sample_image = predict(vgg16_model, new_image_name)
    df_sample_image = pd.DataFrame(sample_image)
    sample_similarity = linear_kernel(df_sample_image, df_embeddings)
    return sample_similarity



def normalize_sim(similarity):
    """ Normalize similarity results """
    x_min = similarity.min(axis=1)
    x_max = similarity.max(axis=1)
    norm = (similarity-x_min)/(x_max-x_min)[:, np.newaxis]
    return norm

sample_similarity_norm = normalize_sim(sample_similarity)
sample_similarity_norm.shape

def get_recommendations(df, similarity):
    """ Return the top 5 most similar products """
    # Get the pairwsie similarity scores of all clothes with that one (index, value)
    sim_scores = list(enumerate(similarity[0]))
    
    # Sort the clothes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 5 most similar clothes
    sim_scores = sim_scores[0:5]
    print(sim_scores)
    # Get the clothes indices
    cloth_indices = [i[0] for i in sim_scores]

    # Return the top 5 most similar products
    return df['image'].iloc[cloth_indices]

recommendation = get_recommendations(styles_df, sample_similarity_norm)
recommendation_list = recommendation.to_list()
#recommended images
plt.figure(figsize=(20,20))
j=0
for i in recommendation_list:
    plt.subplot(6, 10, j+1)
    print(path)
    cloth_img =  mpimg.imread(path + 'images/'+ i)
    plt.imshow(cloth_img)
    plt.axis("off")
    j+=1
plt.title("Recommended images",loc='left')
plt.subplots_adjust(wspace=-0.5, hspace=1)
plt.show()

NameError: name 'new_image_name' is not defined